In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd

from zipfile import ZipFile

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

def addextension(nm):
    return nm+".jpg"

train_df.Image_ID = train_df.Image_ID.apply(addextension)
test_df.Image_ID = test_df.Image_ID.apply(addextension)

print(train_df.head())

def unzip(nm):
    with ZipFile(nm,"r") as zip:
        zip.extractall()

unzip("Train_Images.zip")
unzip("Test_Images.zip")

          Image_ID            class   xmin   ymin  width  height
0  ID_007FAIEI.jpg  fruit_woodiness   87.0   87.5  228.0   311.0
1  ID_00G8K1V3.jpg  fruit_brownspot   97.5   17.5  245.0   354.5
2  ID_00WROUT9.jpg  fruit_brownspot  156.5  209.5  248.0   302.5
3  ID_00ZJEEK3.jpg    fruit_healthy  125.0  193.0  254.5   217.0
4  ID_018UIENR.jpg  fruit_brownspot   79.5  232.5  233.5   182.0


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    horizontal_flip = True,
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 3125 validated image filenames belonging to 3 classes.
Found 781 validated image filenames belonging to 3 classes.


### Model = Transfer ResNet50V2 (Transfer Learning Feature Extraction)

In [4]:
rescale = tf.keras.applications.resnet_v2.preprocess_input
base_model = tf.keras.applications.ResNet50V2(input_shape=(256,256,3),include_top=False,weights="imagenet")
base_model.trainable = False

class Transfer_ResNet50V2():
    def model(self,input):
        self.x = rescale(input)
        self.x = base_model(self.x,training=False)
        self.x = tf.keras.layers.GlobalAveragePooling2D()(self.x)
        self.x = tf.keras.layers.Dropout(0.2,seed=42)(self.x)
        self.output = tf.keras.layers.Dense(3,activation="softmax")(self.x)
        self.model = tf.keras.Model(input,self.output,name="Transfer_ResNet50V2")
        return self.model

TFRES50V2 = Transfer_ResNet50V2()
model = TFRES50V2.model(tf.keras.Input(shape=(256,256,3)))
model.summary()
model.compile(tf.keras.optimizers.RMSprop(),tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])
model.summary()

Model: "Transfer_ResNet50V2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 256, 256, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 8, 8, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 3)         

In [5]:
if __name__=="__main__":
    checkpoint = tf.keras.callbacks.ModelCheckpoint("makerere.hdf5",save_weights_only=False,save_best_only=True,monitor="val_loss")
    model.fit(train_ds,epochs=10,validation_data=val_ds,callbacks=[checkpoint])
    best = tf.keras.models.load_model("makerere.hdf5")
    val_loss,val_acc = best.evaluate(val_ds)
    print("\nVal Accuracy: {:.2f} %".format(100*val_acc))
    print("Val Loss: {:.2f} %".format(100*val_loss))

Epoch 1/10
98/98 [==============================] - 54s 498ms/step - loss: 0.4431 - accuracy: 0.8272 - val_loss: 0.2695 - val_accuracy: 0.9078


/home/maverick/.local/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/10
98/98 [==============================] - 45s 461ms/step - loss: 0.2481 - accuracy: 0.9040 - val_loss: 0.1822 - val_accuracy: 0.9309
Epoch 3/10
98/98 [==============================] - 46s 473ms/step - loss: 0.2017 - accuracy: 0.9302 - val_loss: 0.1508 - val_accuracy: 0.9373
Epoch 4/10
98/98 [==============================] - 46s 471ms/step - loss: 0.1649 - accuracy: 0.9434 - val_loss: 0.1362 - val_accuracy: 0.9501
Epoch 5/10
98/98 [==============================] - 47s 478ms/step - loss: 0.1543 - accuracy: 0.9488 - val_loss: 0.1179 - val_accuracy: 0.9590
Epoch 6/10
98/98 [==============================] - 48s 494ms/step - loss: 0.1316 - accuracy: 0.9507 - val_loss: 0.1339 - val_accuracy: 0.9475
Epoch 7/10
98/98 [==============================] - 48s 493ms/step - loss: 0.1311 - accuracy: 0.9514 - val_loss: 0.1171 - val_accuracy: 0.9629
Epoch 8/10
98/98 [==============================] - 47s 482ms/step - loss: 0.1204 - accuracy: 0.9590 - val_loss: 0.1235 - val_accuracy: 0.9680